# Practicing with Data Science Salaries Dataset
#### Assumption is that the data obtained provides (possibly) an accurate representation of the salaries of data scientist.
#### The objective of this analysis is to identify whether the salaries of data scientist (or data analytics related jobs) are above the average annual salary relative to the Country in which the employee is situated in.

Dataset and description of variables/features can be obtained from: https://salaries.ai-jobs.net/download/

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iso-country-codes-global/wikipedia-iso-country-codes.csv
/kaggle/input/data-science-job-salaries/ds_salaries.csv
/kaggle/input/average-annual-wages-by-country-from-oecd/AV_AN_WAGE_24072022124154184.csv


# Initial Data work and processes

### Loading relevant datasets:
    1. Data Science Salaries
    2. Average Global Salaries
    3. Country codes 

In [2]:
salaries = pd.read_csv('../input/data-science-job-salaries/ds_salaries.csv')
average_global_sal = pd.read_csv("../input/average-annual-wages-by-country-from-oecd/AV_AN_WAGE_24072022124154184.csv")
country_codes = pd.read_csv("../input/iso-country-codes-global/wikipedia-iso-country-codes.csv")

### Starting with Data Science Salaries Dataset

In [3]:
salaries.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [4]:
salaries.describe()

,Unnamed: 0,work_year,salary,salary_in_usd,remote_ratio
count,607.000000,607.000000,6.070000e+02,607.000000,607.00000
mean,303.000000,2021.405272,3.240001e+05,112297.869852,70.92257
std,175.370085,0.692133,1.544357e+06,70957.259411,40.70913
min,0.000000,2020.000000,4.000000e+03,2859.000000,0.00000
25%,151.500000,2021.000000,7.000000e+04,62726.000000,50.00000
50%,303.000000,2022.000000,1.150000e+05,101570.000000,100.00000
75%,454.500000,2022.000000,1.650000e+05,150000.000000,100.00000
max,606.000000,2022.000000,3.040000e+07,600000.000000,100.00000


In [5]:
salaries.info() #No NA values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


In [6]:
salaries.nunique()

Unnamed: 0            607
work_year               3
experience_level        4
employment_type         4
job_title              50
salary                272
salary_currency        17
salary_in_usd         369
employee_residence     57
remote_ratio            3
company_location       50
company_size            3
dtype: int64

Each observation for "Unnamed: 0" column is unique, would not provide much useful information.

In [7]:
salaries.drop("Unnamed: 0", axis = 1, inplace = True)

In [8]:
salaries.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

#### Identifying and changing object type columns into categorical.

In [9]:
salaries.select_dtypes(include = "object").nunique()

experience_level       4
employment_type        4
job_title             50
salary_currency       17
employee_residence    57
company_location      50
company_size           3
dtype: int64

In [10]:
cols_to_cat = salaries.select_dtypes(include = "object").filter(items = ["experience_level", "employment_type", "company_size"]).columns


In [11]:
#Checking the unique values in each object type variable in the dataset
for i in cols_to_cat:
    print(i)
    print(salaries[i].unique())

experience_level
['MI' 'SE' 'EN' 'EX']
employment_type
['FT' 'CT' 'PT' 'FL']
company_size
['L' 'S' 'M']


In [12]:
#Changing the value names into their exact names listed on the website for easier reading
salaries["experience_level"].replace(["EN", "MI", "SE","EX"],
                                     ["Entry-level / Junior", "Mid-level / Intermediate", "Senior-level / Expert", "Executive-level / Director"],
                                    inplace = True)
salaries["employment_type"].replace(['PT', 'FT', 'CT', 'FL'], ["Part-time", "Full-time", "Contract", "Freelance"],
                                    inplace = True)
salaries["company_size"].replace(['S', 'M', 'L'], ["Small", "Medium", "Large"],
                                    inplace = True)
salaries["remote_ratio"].replace([0, 50, 100], ["No Remote", "Partially Remote", "Fully Remote"],
                                    inplace = True)

In [13]:
# ['stack', 'group', 'overlay', 'relative']
fig1 = px.histogram(salaries, x = "work_year",
                    template = "plotly_white",
                    color = "company_location",
                    barmode = "overlay",
                    title = "Observations Based on Years")

fig1.update_xaxes(type='category')

fig1.update_layout(hovermode = "x",
                  yaxis_title = "No. of Observations",
                  xaxis_title = "Year",
                  showlegend = False)
fig1.show()

## Observation Distributions Based On Experience Level, Employment Type and Remote Work 

In [14]:
fig2 = make_subplots(rows=1, shared_yaxes = True, cols=3,column_titles= ["Experience Level Distribution", "Employment Type Distribution", "Remote Work Distribution"])

fig2.add_trace(go.Histogram(x=salaries[cols_to_cat[0]],
                           hovertemplate = '<br>'.join([
                               'No. of observations: %{y}',
                               'Position: %{x}'                               
            ]), name = "Experience"), 1,1)
fig2.add_trace(go.Histogram(x=salaries[cols_to_cat[1]], 
                           name = "Employment",
                           hovertemplate = '<br>'.join([
                               'No. of observations: %{y}',
                               'Employment Type: %{x}'
    
            ])),
              1,2)
fig2.add_trace(go.Histogram(x=salaries["remote_ratio"], name = "Remote Work"),
               1,3)


fig2.update_xaxes(categoryorder="category ascending")
fig2.update_yaxes(showspikes = True)
fig2.update_layout(template = "ggplot2",hovermode = "x")
fig2.update_traces(showlegend = False)
fig2.show()


The Distributions are rather imbalanced based on the current number of observations, most notably in the Employment Type variable. The large imbalanced in the Employment Type variable may possibly be due to the increase in demand for Data Scientist and Data Analytics related professionals to help automate or improve efficiencies within an enterprise. Additionally, other Employment Types may vary significantly depending on the type of projects and industries that they are involved in.

The Experience Level variable will be further expanded upon in the next section. Where we will look at the salary distribution in terms of different Experience Levels and Company Size.

In [15]:
fig3 = px.bar(salaries.groupby(["company_size", "experience_level"]).mean().reset_index(),    
               x = "experience_level",
               y = "salary_in_usd",
              color = "company_size",
              text_auto='.2s',              
              title = "Global Average Salary (USD) Based on Experience Level & Company Size")

fig3.update_traces(textposition = "outside",
                   hovertemplate = '<br>'.join([
                       'Salary: %{y}'
                   ]))
fig3.update_layout(barmode = 'group',
#                    xaxis = {'categoryorder' :'array','categoryarray' :['Small','Medium','Large']},
                   title_font_color="#085CB0",
                   font_color="#085CB0",
                   yaxis_title="Salary (USD)",
                   xaxis_title="Experience Level",
                   legend_title = "Company Sizes",
                   hovermode = "x",
                   template = "ggplot2",
                   legend_traceorder="reversed")


                   
fig3.update_xaxes(categoryorder='array', 
                  categoryarray= ["Entry-level / Junior", "Mid-level / Intermediate", "Senior-level / Expert", "Executive-level / Director"])
fig3.show()

In [16]:
fig4 = px.scatter(salaries, x ="salary_in_usd", y= "experience_level", size = "salary_in_usd", color = "company_size",
                  title = "Salary Distribution Based On Experience Levels & Company Sizes", template = "seaborn")
fig4.update_layout(
    yaxis_title="Experience Level",
    xaxis_title="Salary (USD)",
    legend_title = "Company Sizes:",
    hovermode = "x",
    legend_traceorder="grouped",
    legend=dict(
    orientation="h"))

fig4.update_yaxes(categoryorder='array', 
                  categoryarray= ["Entry-level / Junior", "Mid-level / Intermediate", "Senior-level / Expert", "Executive-level / Director"])
fig4.show()

In [17]:
fig5 = px.box(salaries, y = "salary_in_usd", color = "experience_level", facet_col = "company_size", template = "plotly_dark")
fig5.update_yaxes(categoryorder='total ascending')
fig5.update_layout(hovermode = "x",
                  legend_traceorder="reversed")
fig5.show()
#The distribution of salary is more stable for medium-sized companies.

Snippet of the data description:

"The experience level in the job during the year with the following possible values:
EN: Entry-level / Junior
MI: Mid-level / Intermediate
SE: Senior-level / Expert
EX: Executive-level / Director"

The dataset contains more information on the salaries of Senior-level Data Scientist (DS) with Mid-level DS being second.
Based on Salary Distribution Based On Experience Levels & Company Sizes graph, we see that medium sized firms make up most of the observations, specifically in the Mid-level and Senior-levels. Interestingly, small sized firms tend to pay a higher amount for most of the Experience Levels, besides Mid-level positions.

As expected, large-sized firms provide the highest average salary (USD) for all four levels of experience. However, we can actually separate the four experience levels into two groups which are:
1. Entry-level and Executive-level DS
2. Mid-level and Senior-level level DS

Their relationships between company size and average salary show differing contrast between the two groups, in which will be discussed further.

For **Entry-Level and Executive-level DS**, the distribution of salaries does not have a significant relationship with company size, based on the graph above. Large-sized companies still provide the highest average salary, however, the salary paid by small companies for both entry  and executive level are **higher than those in the mid-sized companies**.

For **Mid-level and Senior-level level DS**, on average, the increase in pay reflects the increase of the company size. SE level DS already command a 
high pay due to their seniority and experience, hence the reason for the pay jump from the different company sizes increases gradually.

In contrast, the increase in pay for a small sized company to medium size company shows a more significant pay jump from an average salary of USD51,000 to USD90,000.

Additionally, it should be noted as well that the pay of an Senior-lvel in a small firm is still higher than an Mid-lvel at a large sized company.




## Employee Residence 

### Understanding the demographic of observations

#### Will be using "work_year" as the "counts" column in this initial phase.

In [18]:
countries_count = salaries.groupby("employee_residence").count().reset_index()
countries_btm = countries_count.query("work_year <= 10")
countries_count.loc[countries_count['work_year'] <= 10, 'employee_residence'] = 'Others'

In [19]:
top_countries = countries_count.query("work_year > 10")
fig6 = px.bar(top_countries[['employee_residence', 'work_year']],
              x = "employee_residence", y = "work_year",
               text_auto='.2s',
              title = "Top Countries with more than 10 observations"
             )
fig6.update_xaxes(categoryorder='total descending')
# fig3.update_layout(xaxis={'categoryorder':'total descending'})

Majority of respondents are from the United States. 

## Obtaining The Average Annual Salary For Each Country
### Important Note: The dataset only contains up until 2021. Hence moving forward, the dataset period will be for 2020 to 2021 only.


In [20]:
average_global_sal["Series"].unique()

array(['Current prices in NCU', '2021 constant prices and NCU',
       'In 2021 constant prices at 2021 USD PPPs'], dtype=object)

#### Filtering only for columns that show the values in USD for each country's average annual salary.

In [21]:
average_global_sal = average_global_sal[(average_global_sal["Unit Code"] == "USD") & (average_global_sal["Series"] == "In 2021 constant prices at 2021 USD PPPs")][[
    "COUNTRY", "Country", "Time", "Value", "Unit Code"
]].copy()
average_global_sal.rename(columns = {"COUNTRY": "Alpha-3 code"}, inplace = True)
average_global_sal.head()

,Alpha-3 code,Country,Time,Value,Unit Code
1276,AUS,Australia,2000,45961.871253,USD
1277,AUS,Australia,2001,46424.274156,USD
1278,AUS,Australia,2002,46814.171091,USD
1279,AUS,Australia,2003,47450.472102,USD
1280,AUS,Australia,2004,48917.552922,USD


#### Renaming columns for ease of merging later on.

In [22]:
country_codes.rename(columns = {"English short name lower case": "Country"}, inplace = True)
country_codes2 = country_codes[["Country", "Alpha-2 code", "Alpha-3 code"]].copy().reset_index()
average_global_sal.columns

Index(['Alpha-3 code', 'Country', 'Time', 'Value', 'Unit Code'], dtype='object')

#### Merging average global salary with country codes dataset

In [23]:
average_global_sal2 = average_global_sal.merge(country_codes2, on = "Alpha-3 code")
average_global_sal2 = average_global_sal2[["Country_x", "Country_y", "Time", "Value", "Alpha-2 code"]]

In [24]:
average_global_sal2.head()

,Country_x,Country_y,Time,Value,Alpha-2 code
0,Australia,Australia,2000,45961.871253,AU
1,Australia,Australia,2001,46424.274156,AU
2,Australia,Australia,2002,46814.171091,AU
3,Australia,Australia,2003,47450.472102,AU
4,Australia,Australia,2004,48917.552922,AU


#### Check whether the countries match by using "test" column to identify the columns that do not match

In [25]:
average_global_sal2["test"] = average_global_sal2["Country_x"] == average_global_sal2["Country_y"]
average_global_sal2[average_global_sal2["test"] == False][["Country_x", "Country_y"]].value_counts()

Country_x        Country_y  
Korea            South Korea    22
Slovak Republic  Slovakia       22
dtype: int64

#### Choosing the relevant variables.

In [26]:
complete_salary = salaries.merge(average_global_sal2, left_on = "company_location", right_on = "Alpha-2 code")
complete_salary.rename(columns = {"Country_x": "Country"}, inplace = True)
completed_salary = complete_salary[["work_year", "Time", "experience_level", "employment_type", "job_title", "salary_in_usd",
                                   "Value","company_size", "Country"]]

In [27]:
completed_salary.head()

,work_year,Time,experience_level,employment_type,job_title,salary_in_usd,Value,company_size,Country
0,2020,2000,Mid-level / Intermediate,Full-time,Data Scientist,79833,47710.984747,Large,Germany
1,2020,2001,Mid-level / Intermediate,Full-time,Data Scientist,79833,48058.921436,Large,Germany
2,2020,2002,Mid-level / Intermediate,Full-time,Data Scientist,79833,48342.057817,Large,Germany
3,2020,2003,Mid-level / Intermediate,Full-time,Data Scientist,79833,48502.276451,Large,Germany
4,2020,2004,Mid-level / Intermediate,Full-time,Data Scientist,79833,48496.021661,Large,Germany


#### Filtering only for years 2020 and 2021 (the latest data from average annual salary dataset)

In [28]:
completed_salary2 = completed_salary[(completed_salary["Time"] >= 2020) & (completed_salary["Time"] == completed_salary["work_year"])]

In [29]:
#Creating a new variable that categorises whether the employee's salary is above the annual average salary of their particular country.
completed_salary2["salary_rank"] = completed_salary2.apply(lambda x: "Above Average" if x["salary_in_usd"] > x["Value"] else "Below Average", axis = 1).copy()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Grouping the dataset by Country, Experience Level, Salary Rank (Above or Below Average) and Time (Year).

In [30]:
group_sal = completed_salary2.groupby([ "Country", "experience_level", "salary_rank", "company_size", "Time"], as_index = False).mean()

In [31]:
group_sal.dtypes

Country              object
experience_level     object
salary_rank          object
company_size         object
Time                  int64
work_year           float64
salary_in_usd       float64
Value               float64
dtype: object

## Analyzing The Difference in Salaries of Respondents And Their Respective Domestic Average Annual Salary For The Year 2020 and 2021 
#### Above And Below Average Distribution For Different Experience Levels

In [32]:
fig8 = px.scatter(group_sal, x = "salary_in_usd", 
                  y = "experience_level",template = "ggplot2",
                  color = "salary_rank", size = "Value",
                  facet_col = "Time",
                  animation_frame = "company_size",
                  title = "Salary Comparison To Country Average")
# fig1.update_yaxes(type = "category")

fig8.update_xaxes(showspikes = True,
                  spikemode  = 'across+marker',
                  spikesnap = 'cursor',
                  showline=True,
                  showgrid=True)

for axis in fig8.layout:
    if type(fig8.layout[axis]) == go.layout.XAxis:
        fig8.layout[axis].title.text = 'Salary (USD)'
        
fig8.update_yaxes(categoryorder='array', 
                  categoryarray= ["Entry-level / Junior", "Mid-level / Intermediate", "Senior-level / Expert", "Executive-level / Director"])

fig8.update_layout(hovermode = "x",
                   yaxis_title = "Experience Level")

fig8.show()

## WORK-IN-PROGRESS
#### Title says it all.

In [33]:

fig7 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig7.add_trace(go.Pie(labels=group_sal[group_sal["Time"] == 2020]["salary_rank"], name="2020 Ratio",),
              1, 1)
fig7.add_trace(go.Pie(labels=group_sal[group_sal["Time"] == 2021]["salary_rank"], name="2021 Ratio"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig7.update_traces(hole=.4, hoverinfo="label+percent+name")

fig7.update_layout(
    title_text="Data Science Salaries Above Annual Average Salary For Years 2020 and 2021",
    # Add annotations in the center of the donut pies.
    template = "seaborn",
    annotations=[dict(text='2020', x=0.2, y=0.5, font_size=20, font_color = "#72AACA",showarrow=False),
                 dict(text='2021', x=0.8, y=0.5, font_size=20, font_color = "#C6CD60", showarrow=False)])

fig7.show()

It can be seen that there is a slight increase in terms of salaries being above the domestic average annual salary. However, this can be due to multiple other factors such as:

    1. A Higher number of Senior-level and/or Executive-Level respondents to the dataset that has brought up the percentage of "Above Average" observations in dataset for 2021, and

    2. An increase in demand but a slowly increase in supply for credible and experienced Data Scientist/Data Analyst may be leading to higher salaries advertised to attract .

Next step is to include a slider or dropdown to go through the different countries/continents percentage difference of above and below average salaries for the two years.

Hoping to possibly expand this later on as the dataset increases and the domestic average annual salary for 2022 is released much later on.

In [34]:
group_sal3 = completed_salary2.groupby([ "Country", "salary_rank", "company_size", "Time"], as_index = False).mean()
group_sal3

,Country,salary_rank,company_size,Time,work_year,salary_in_usd,Value
0,Austria,Above Average,Large,2020,2020.0,74130.00,56807.491932
1,Austria,Above Average,Medium,2021,2021.0,61467.00,58138.589524
2,Austria,Above Average,Small,2020,2020.0,91237.00,56807.491932
3,Belgium,Above Average,Medium,2021,2021.0,85699.00,59601.499049
4,Canada,Above Average,Large,2020,2020.0,117104.00,56257.713946
...,...,...,...,...,...,...,...
77,United States,Below Average,Large,2021,2021.0,50232.25,74737.845354
78,United States,Below Average,Medium,2020,2020.0,56000.00,72806.754008
79,United States,Below Average,Medium,2021,2021.0,42600.00,74737.845354
80,United States,Below Average,Small,2020,2020.0,52880.00,72806.754008


In [35]:
px.bar(group_sal3, "salary_in_usd", y = "Country", facet_col = "Time", animation_frame = "company_size", template = "plotly_dark")